При исследовании данного датасета я произвел одну полную замену выбросов в столбце Age, посчитав, что поскольку в данном столбце есть значения только 9 и 1 (взрослых лошадей), и совсем нет 2 (жеребят), то это явно какая-то ошибка ввода. Поэтому заменил 9 на 2. С этим же делением на взрослых особей и жеребят произвел заполнение пропусков в Pulse медианой с группировкой по Age - у больных жеребят и взрослых особей слишком разные показатели пульса и применять для всех усредненные значения мне показалось неправильным. 
И, несмотря на рекомендации не делать так в домашнем задании, я все-таки удалил одну строку. В ней одновременно были единственный пропуск по Surgery и единственный пропуск по Outcome. Т.е. слишком много непонятного в этой строке - то ли лошадь оперировали, то ли нет, то ли она умерла в итоге, то ли осталась жива. Слишком много разных вариантов может быть, а значит усреднять и делать какие-то выводы по этой строке довольно опасно, поэтому данную строку я решил удалить.
Остальное исследование ниже.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded= files.upload()

Saving horse_data.csv to horse_data.csv


#**Задание 1. Базовое изучение**

Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [3]:
df = pd.read_csv('horse_data.csv', header = None, na_values = '?')
df1 = df[[(0), (23), (22), (1), (3), (13), (4), (19)]]
df1.columns = ['Surgery', 'Surgical lesion', 'Outcome', 'Age', 'Rectal temper', 'nasogastric tube', 'Pulse', 'Total protein']
df1





,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,NaN,66.0,8.4
1,1.0,2,3.0,1,39.2,NaN,88.0,85.0
2,2.0,2,1.0,1,38.3,NaN,40.0,6.7
3,1.0,1,2.0,9,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,NaN,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,NaN,NaN,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,NaN
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0


# Трактовка средних значений

Среднее значение ректальной температуры - 38.17, что выше температуры здоровой лошади (~37,5). Среднее значение пульса - 71,9, что также выше нормальных значений пульса. Средние значения общего белка - 24.46 gms/dL сильно превышают нормальные значения (6-7,5). Повышенные средние значения говорят о том, что большая часть лошадей больны.

In [4]:
df1.describe()

,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
count,299.000000,300.000000,299.000000,300.000000,240.000000,196.000000,276.000000,267.000000
mean,1.397993,1.363333,1.551839,1.640000,38.167917,1.755102,71.913043,24.456929
std,0.490305,0.481763,0.737187,2.173972,0.732289,0.649437,28.630557,27.475009
min,1.000000,1.000000,1.000000,1.000000,35.400000,1.000000,30.000000,3.300000
25%,1.000000,1.000000,1.000000,1.000000,37.800000,1.000000,48.000000,6.500000
50%,1.000000,1.000000,1.000000,1.000000,38.200000,2.000000,64.000000,7.500000
75%,2.000000,2.000000,2.000000,1.000000,38.500000,2.000000,88.000000,57.000000
max,2.000000,2.000000,3.000000,9.000000,40.800000,3.000000,184.000000,89.000000


# Трактовка модальных значений

Моды категориальных значений говорят о том, что большинство лошадей перенесло хирургическое вмешательство и большинство проблем носили хирургический характер. Чаще всего после хирургического вмешательства животное продолжало жить. Чаще колики испытывают взрослые лошади. Назогастральная трубка в основном имеет незначительные значения. В большинстве случев вздутия живота не наблюдается.

In [111]:
print('Моды: \n Surgery: ', df1['Surgery'].round().mode()[0], '\n Surgical lesion:', df1['Surgical lesion'].round().mode()[0], '\n Outcome:', df1['Outcome'].round().mode()[0], '\n Age:', df1['Age'].round().mode()[0], '\n nasogastric tube:', df1['nasogastric tube'].round().mode()[0] ) 

Моды: 
 Surgery:  1.0 
 Surgical lesion: 1 
 Outcome: 1.0 
 Age: 1 
 nasogastric tube: 2.0




---


#**Задание 2. Работа с выбросами** 
В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

#2.1 Surgery/Хирургия


In [113]:
q1 = df1['Surgery'].quantile(0.25)
q3 = df1['Surgery'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Surgery'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
print('Выбросов: ', len(remove_outliers['Surgery']))
remove_outliers['Surgery']

Выбросов:  1


132   NaN
Name: Surgery, dtype: float64

# 2.2 Surgical lesion/Имела ли проблема хирургический способ решения

Выбросов и пропусков нет

In [114]:
q1 = df1['Surgical lesion'].quantile(0.25)
q3 = df1['Surgical lesion'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Surgical lesion'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
print('Выбросов: ', len(remove_outliers['Surgical lesion']))
remove_outliers['Surgical lesion']

Выбросов:  0


Series([], Name: Surgical lesion, dtype: int64)

# 2.3 Outcome/Исход

In [115]:
q1 = df1['Outcome'].quantile(0.25)
q3 = df1['Outcome'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Outcome'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
remove_outliers['Outcome']

132   NaN
Name: Outcome, dtype: float64

#2.4 Age/Возраст

Найдено 24 выброса одинакового значения "9". Можно было бы проинтерпертировать это следующим образом - скорее всего кто-то забыл об условии деления "взрослая - молодая лошадь" (1 или 2) и писал просто возраст лошади в годах. Но тогда получается, что все 24 лошади одинакового возраста - 9 лет. На мой взгляд, шансы такого совпадения малы, лошади подвержены коликам в любом возрасте.
Значит, это какая-то ошибка.
Если отсоритровать значения столбца Age в порядке убывания, то мы видим, что у нас среди значений есть только "9" и "1", а "2" нет, хотя логично было бы предположить, что они есть, должны же попадться жеребята. Делаю предположение о том, что кто-то вместо двоек просто по ошибке везде написал девятки и принимаю решение о замене девяток двойками.

1. Нахожу выбросы

In [116]:
q1 = df1['Age'].quantile(0.25)
q3 = df1['Age'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Age'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
print('Выбросов: ', len(remove_outliers['Age']))
remove_outliers['Age']

Выбросов:  24


3      9
9      9
13     9
16     9
23     9
39     9
41     9
55     9
74     9
75     9
82     9
103    9
109    9
135    9
191    9
212    9
227    9
229    9
244    9
252    9
255    9
262    9
275    9
287    9
Name: Age, dtype: int64

2. Обнаруживаю, что у нас есть только девятки и единицы

In [117]:
sorted_df = df1.sort_values(by='Age', ascending=False)
sorted_df.head(30)

,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
191,2.0,2,1.0,9,NaN,NaN,100.0,4.7
41,2.0,1,1.0,9,39.0,NaN,150.0,8.5
275,1.0,1,2.0,9,38.8,1.0,150.0,6.2
135,1.0,1,2.0,9,38.1,2.0,136.0,4.9
109,1.0,1,2.0,9,38.4,1.0,84.0,6.6
103,1.0,1,1.0,9,38.0,NaN,140.0,5.3
23,1.0,1,1.0,9,38.3,NaN,130.0,70.0
212,1.0,2,1.0,9,38.9,3.0,120.0,6.3
39,1.0,1,2.0,9,39.2,NaN,146.0,NaN
74,1.0,1,2.0,9,NaN,NaN,NaN,4.9


3. Заменяю "9" на "2". Теперь похоже на правду.

In [118]:
df1.loc[(df1['Age'] == 9), 'Age'] = 2
df1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,NaN,66.0,8.4
1,1.0,2,3.0,1,39.2,NaN,88.0,85.0
2,2.0,2,1.0,1,38.3,NaN,40.0,6.7
3,1.0,1,2.0,2,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,NaN,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,NaN,NaN,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,NaN
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0


#2.5 Rectal temper/Ректальная температура

С помощью max() проверил максимальное значение выброса - 40.8, минимальное - 35.4
Это значит что среди выбросов нет каких-то слишком больших значений, и все их вполне можно считать вполне вероятными показателями для больной лошади. Так что выбросами их не считаем и берем в работу.



In [119]:
q1 = df1['Rectal temper'].quantile(0.25)
q3 = df1['Rectal temper'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Rectal temper'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
remove_outliers['Rectal temper']

5       NaN
7       NaN
8       NaN
16      NaN
20     39.9
       ... 
274     NaN
281    40.0
293     NaN
295     NaN
298    36.5
Name: Rectal temper, Length: 66, dtype: float64

#2.6 Nasogastric tube/Назогастральная трубка

Выбросы - все значения Nan

In [120]:
q1 = df1['nasogastric tube'].quantile(0.25)
q3 = df1['nasogastric tube'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['nasogastric tube'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
print('Выбросов: ', len(remove_outliers['nasogastric tube']))
remove_outliers['nasogastric tube']

Выбросов:  102


0     NaN
1     NaN
2     NaN
4     NaN
17    NaN
       ..
289   NaN
290   NaN
294   NaN
295   NaN
299   NaN
Name: nasogastric tube, Length: 102, dtype: float64

#2.7 Pulse/Пульс

Максимальное значение выброса - 184 уд./мин.- это слишком высокий показатель для взрослой лошади. Но вполне возможный показатель для больного жеребенка. И действительно, если посмотреть просто remove_outliers, то увидим, то у всех жеребят пульс гораздо выше взрослых особей.
Поэтому не считаем выбросы какими-то неправильными значениями и оставляем их

In [121]:
q1 = df1['Pulse'].quantile(0.25)
q3 = df1['Pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Pulse'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
#remove_outliers
remove_outliers['Pulse']



3      164.0
5        NaN
28       NaN
41     150.0
52       NaN
55     160.0
56       NaN
58       NaN
74       NaN
93       NaN
115      NaN
117      NaN
126      NaN
151      NaN
159      NaN
160      NaN
174      NaN
204      NaN
207      NaN
216      NaN
227      NaN
255    184.0
275    150.0
Name: Pulse, dtype: float64

#2.8 Total protein/Общий белок

In [122]:
q1 = df1['Total protein'].quantile(0.25)
q3 = df1['Total protein'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df1[df1['Total protein'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([df1, remove_outliers]).drop_duplicates(keep=False)
print('Выбросов: ', len(remove_outliers['Total protein']))
remove_outliers['Total protein']

Выбросов:  29


5     NaN
17    NaN
25    NaN
28    NaN
39    NaN
51    NaN
55    NaN
59    NaN
72    NaN
81    NaN
101   NaN
113   NaN
130   NaN
137   NaN
151   NaN
158   NaN
169   NaN
174   NaN
177   NaN
185   NaN
216   NaN
218   NaN
220   NaN
230   NaN
243   NaN
245   NaN
248   NaN
274   NaN
296   NaN
Name: Total protein, dtype: float64



---


#**Задание 3. Работа с пропусками**
Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

#3.1 
В столбцах "Surgical lesion", "Age" пропусков нет

#3.2 
Как видим в "Surgery" и "Outcome" есть единственный пропуск и он в строке с id 132. Данную строку я бы просто удалил, потому что вряд ли при определении исхода уместна замена каким-то модальным значением. Одновременно нет понимания, было ли хирургическое вмешательство и скончалась ли в итоге лошадь, и тут довольно тяжело угадать или заменить каким-то подходящим значением. Поэтому хоть удалять и не рекомендуется, но иначе слишком уж неопределенная запись получается.
Удаляем, таким образом "Surgery" и "Outcome" становятся без пропусков

In [162]:
df2 = df1.drop(df1.index[132])
df2

,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,NaN,66.0,8.4
1,1.0,2,3.0,1,39.2,NaN,88.0,85.0
2,2.0,2,1.0,1,38.3,NaN,40.0,6.7
3,1.0,1,2.0,2,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,NaN,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,NaN,NaN,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,NaN
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0


#3.3 Ректальная температура

Поскольку в зависимости от стадии болезни температура животного может быть как выше нормы, так и ниже, то считаю, что пропуски в данном столбце нужно заменить средним значением.

In [163]:
df2['Rectal temper'].fillna(df2['Rectal temper'].median(), inplace=True)
df2

,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,NaN,66.0,8.4
1,1.0,2,3.0,1,39.2,NaN,88.0,85.0
2,2.0,2,1.0,1,38.3,NaN,40.0,6.7
3,1.0,1,2.0,2,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,NaN,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,38.2,NaN,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,NaN
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0


#3.4 Назогастральная трубка

Не очень понятно, насколько сильно этот показатель влияет на остальные, поэтому я бы заменил его модой.

In [164]:
df2['nasogastric tube'].fillna(df2['nasogastric tube'].mode()[0], inplace=True)
df2

,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,2.0,66.0,8.4
1,1.0,2,3.0,1,39.2,2.0,88.0,85.0
2,2.0,2,1.0,1,38.3,2.0,40.0,6.7
3,1.0,1,2.0,2,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,2.0,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,38.2,2.0,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,NaN
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0


#3.5 Пульс

Поскольку у больных жеребят и взрослых особей значения пульса различаются слишком сильно, то считаю нужным применить заполнение пропусков медианой с группировкой по Age

In [165]:
df2['Pulse'].fillna(df2.groupby('Age')['Pulse'].transform('median'), inplace=True)
df2

,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,2.0,66.0,8.4
1,1.0,2,3.0,1,39.2,2.0,88.0,85.0
2,2.0,2,1.0,1,38.3,2.0,40.0,6.7
3,1.0,1,2.0,2,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,2.0,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,38.2,2.0,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,NaN
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0


#3.6 Total protein/Общий белок

Считаю, что пропуски нужно заполнить медианным значением

In [166]:
df2['Total protein'].fillna(df2['Total protein'].median(), inplace=True)
df2


,Surgery,Surgical lesion,Outcome,Age,Rectal temper,nasogastric tube,Pulse,Total protein
0,2.0,2,2.0,1,38.5,2.0,66.0,8.4
1,1.0,2,3.0,1,39.2,2.0,88.0,85.0
2,2.0,2,1.0,1,38.3,2.0,40.0,6.7
3,1.0,1,2.0,2,39.1,1.0,164.0,7.2
4,2.0,2,2.0,1,37.3,2.0,104.0,7.4
...,...,...,...,...,...,...,...,...
295,1.0,2,3.0,1,38.2,2.0,120.0,65.0
296,2.0,1,3.0,1,37.2,3.0,72.0,7.5
297,1.0,1,2.0,1,37.5,2.0,72.0,6.8
298,1.0,1,1.0,1,36.5,3.0,100.0,6.0
